# Binary Treee
- Piotr Tylczyński
- Mateusz Kępa

### Import data
We use Titanic data provided by PUT

In [16]:
import copy
import math

import pandas as pd

# data_path = 'titanic-homework.csv'
data_path = 'titanic-homework.csv'
data = pd.read_csv(
    data_path,
)

### Show data

In [17]:
print(data)

    PassengerId  Pclass                                               Name  \
0             1       3                            Braund, Mr. Owen Harris   
1             2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2             3       3                             Heikkinen, Miss. Laina   
3             4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4             5       3                           Allen, Mr. William Henry   
..          ...     ...                                                ...   
95           96       3                        Shorney, Mr. Charles Joseph   
96           97       1                          Goldschmidt, Mr. George B   
97           98       1                    Greenfield, Mr. William Bertram   
98           99       2               Doling, Mrs. John T (Ada Julia Bone)   
99          100       2                                  Kantor, Mr. Sinai   

       Sex  Age  SibSp  Parch  Survived  
0     male   22      

### Pre-purify
Some vales bring us no information as they are more label like one.
This can be seen in *Name* column, which we will remove for brevity of further code.

Also we discretize here age value

In [18]:
data = data.drop(columns='Name')

### Discretize
Simple way with fixed intervals

In [4]:
for idx, row in data.iterrows():
    if data.loc[idx, 'Age'] <= 20:
        data.loc[idx, 'Age'] = 'young'
    elif data.loc[idx, 'Age'] > 20 and data.loc[idx, 'Age'] <= 40:
        data.loc[idx, 'Age'] = 'middle'
    elif data.loc[idx, 'Age'] > 40:
        data.loc[idx, 'Age'] = 'old'

### General entropy function
$$ En(X) = \sum_{a = 1}^{i} P(x_i) \log{P(x_i)} $$

In [5]:
import math
def entropy(data: pd.DataFrame) -> float:
    decision_classes = data.groupby('Survived').count()
    e = 0
    for c in decision_classes.iloc[:,0]:
        p = c / len(data)
        e += (-1) * math.log2(p) * p
    return e

### Conditional entropy

$$ En(X|A) = \sum_{a=1}^{i} \frac{|x_i|}{|X|}En(x_i) $$

In [6]:
def conditional_entropy(data: pd.DataFrame, col_name: str):
    data_reduced: pd.DataFrame = data[[col_name, 'Survived']]
    var_classes = data_reduced.groupby(col_name).count()
    e = 0
    for idx, row in var_classes.iterrows():
        p = row['Survived'] / len(data)
        data_filtered = data_reduced[data_reduced.loc[:, col_name] == idx]
        e += p * entropy(data_filtered)
    return e

### Gain function after choosing variable A

$$ gain(S,A) = En(S) - En(S | A) $$

In [7]:
def gain(data: pd.DataFrame, column_name: str):
    e_before = entropy(data)
    e_after = conditional_entropy(data, column_name)
    return e_before - e_after

### Intrinsic function

$$ II(S, A) = - \sum_{i=1}^{n}\frac{|S_i|}{|S|} \log_2{\frac{|Si|}{|S|}} $$

In [8]:
def intrinsicInfo(data: pd.DataFrame, column_name: str):
    data_reduced = data[[column_name, 'Survived']]
    var_classes = data_reduced.groupby(column_name).count()
    e = 0
    for idx, row in var_classes.iterrows():
        p = row.iloc[0] / len(data_reduced)
        e += (-1) * p * math.log2(p)
    return e

### Gain Ratio

$$ GainRatio(S, A) = \frac{Gain(S,a)}{IntrinsicInfo(S,A)} $$

In [9]:
def gainRatio(data: pd.DataFrame, column_name: str):
    return gain(data, column_name) / intrinsicInfo(data, column_name)

### Create tree

In [30]:
import copy

def create_tree(data: pd.DataFrame, available_attr: []):
    if entropy(data) == 0:
        return list(data.Survived)[0]
    best_attr = available_attr[0]
    best_gain = 0
    local_tree = {}
    for attr in available_attr:
        if attr == 'PassengerId':
            continue
        if attr == 'Age':
            gain = age_spliter(data)[0]
        else:
            gain = gainRatio(data, attr)
        if gain > best_gain:
            best_gain = gain
            best_attr = attr
    if best_attr == 'Age':
        data.sort_values('Age', inplace=True)
        splitter = age_spliter(data)[1]
        available_attr.remove(best_attr)
        sub_tree1 = create_tree(
                data[:splitter],
                copy.deepcopy(available_attr)
            )
        local_tree['(age < ' + str(splitter) + ')'] = sub_tree1
        sub_tree2 = create_tree(
                data[splitter:],
                copy.deepcopy(available_attr)
            )
        local_tree['(age >= ' + str(splitter) + ')'] = sub_tree2
    else:
        data_reduced = data[[best_attr, 'Survived']]
        var_classes = data_reduced.groupby(best_attr).count()
        available_attr.remove(best_attr)
        for aClass, _ in var_classes.iterrows():
            sub_tree = create_tree(
                data[data.loc[:, best_attr] == aClass],
                copy.deepcopy(available_attr)
            )
            local_tree[str(aClass) + '(' + best_attr + ')'] = sub_tree
    return local_tree

tree = create_tree(data, list(data.columns[:-1]))

/tmp/ipykernel_22320/804759315.py:2: RuntimeWarning: invalid value encountered in double_scalars
  return gain(data, column_name) / intrinsicInfo(data, column_name)


### Print tree

In [31]:
import pptree as ppt

def bfs(tree: {}, this_node: ppt.Node):
    if not isinstance(tree, dict):
        return ppt.Node(tree, this_node)
    for key in tree.keys():
        next_node = ppt.Node(key, this_node)
        bfs(tree[key], next_node)
    return this_node

ppt.print_tree(bfs(tree, ppt.Node('Toot', None)))

                ┌1(Pclass)┐
                │         └1
                ├2(Pclass)┐
                │         └1
     ┌(age < 23)┤
     │          │         ┌male(Sex)┐
     │          │         │         └0
     │          └3(Pclass)┤
     │                    │           ┌0(SibSp)┐
     │                    │           │        └1
     │                    │           ├2(SibSp)┐
     │                    │           │        └0
     │                    │           ├3(SibSp)┐
     │                    │           │        └0
     │                    └female(Sex)┤
     │                                │        ┌0(Parch)┐
     │                                │        │        └0
     │                                ├1(SibSp)┤
     │                                │        └1(Parch)┐
     │                                │                 └1
     │                                ├5(SibSp)┐
     │                                │        └0
     │                                └4(Sib

### Age spliter

In [29]:
def age_spliter(data: pd.DataFrame):
    data_reduced = data[['Age', 'Survived']]
    data_reduced.sort_values('Age', inplace=True)
    divider_pos = 1
    best_entropy = 1
    best_pos = 1
    while divider_pos < len(data):
        el = (divider_pos / len(data)) * entropy(data_reduced.iloc[:divider_pos])
        ep = (len(data) - divider_pos) / len(data) * entropy(data_reduced.iloc[divider_pos:])
        total = el + ep
        if total < best_entropy:
            best_entropy = total
            best_pos = divider_pos
        divider_pos += 1
    return best_entropy, best_pos